### HT table results for excitatory types

In [1]:
source("./scale.R")
test <- read.csv("/data/sag_data.csv")
options(warn=-1)

Loading required package: spatstat.data

Loading required package: spatstat.geom

spatstat.geom 3.2-9

Loading required package: spatstat.random

spatstat.random 3.2-3

Loading required package: spatstat.explore

Loading required package: nlme

spatstat.explore 3.2-7

Loading required package: spatstat.model

Loading required package: rpart

spatstat.model 3.2-11

Loading required package: spatstat.linnet

spatstat.linnet 3.1-5


spatstat 3.0-8 
For an introduction to spatstat, type ‘beginner’ 



Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
seeds <- sample(1:100,5,replace=FALSE)
seeds

[1] 25 42 69 17 61

In [3]:
library('foreach')
library('doParallel')
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

Loading required package: iterators

Loading required package: parallel



### HT table calculation

In [4]:
ecdf_fun <- function(x,perc) ecdf(x)(perc)
## Calculate column stats for each type
calc_pp_2d<-function(typ,pp,ps,section){
    nnp <- c()
    nni <- c()
    mnn <- c()
    res<- list()
    for (i in 1:length(pp)){     
        perc<- ecdf_fun(nndist(pp[[i]]), 0.01)
        int<- intensity(pp[[i]])
        mval<- mean(marks(pp[[i]]))
        nnp<-append(nnp,perc)
        nni<-append(nni,int)
        mnn <- append(mnn, mval)
        res[[as.character(section[i])]] = min(ps[,i])    
    }   
    pval <- signif(fisher(unname(unlist(res)))$p,2)
    res[['pval']] = pval
    res[['perc']] = round(mean(nnp)*100,2)
    res[['lam']] = round(mean(nni))
    res[['conf']] = round(mean(mnn),2)
    res[['rm']] = length(ps[,1])
    res[['Cluster']] = typ
    return(res)
}

### Excitatory clusters
We get the list of types and sections from the filtered table.  
We seperately test for 0074 and 0109.

In [10]:
df<- read.csv("./temp/filter-exc.csv")
df1 <- replace(df, is.na(df), 0)
df1[df1<=0.5] <- NA
df1<-df1[rowSums(is.na(df1)) != ncol(df1)-1, ]
df1<-df1[!df1$cluster%in%(c('0074 L4/5 IT CTX Glut_1','0109 L2/3 IT CTX Glut_2')), ]
row.names(df1) <- NULL

In [11]:
types = df1$cluster
types

[1] "0003 CLA-EPd-CTX Car3 Glut_1" "0004 CLA-EPd-CTX Car3 Glut_1"
 [3] "0007 IT EP-CLA Glut_1"        "0026 L5/6 IT TPE-ENT Glut_2" 
 [5] "0061 L5 IT CTX Glut_3"        "0079 L4/5 IT CTX Glut_2"     
 [7] "0086 L4/5 IT CTX Glut_3"      "0092 L4/5 IT CTX Glut_4"     
 [9] "0114 L2/3 IT CTX Glut_3"      "0116 L2/3 IT CTX Glut_3"     
[11] "0359 L5 ET CTX Glut_2"        "0375 L5 ET CTX Glut_5"       
[13] "0439 L6 CT CTX Glut_1"        "0443 L6 CT CTX Glut_2"       
[15] "0453 L6 CT CTX Glut_5"        "0455 L6 CT CTX Glut_5"

In [12]:
sections<- apply(df1[,-1], 1, function(i) colnames(df1[,-1])[ !is.na(i) ]) ## qualified colns
sections<- sapply(sections, function(x){as.numeric(gsub("X.", "", x))})

In [8]:
df = data.frame(matrix(vector(), 0, 11,
                dimnames=list(c(), c('Cluster','24','23','22','21','20',"pval", "perc", "lam",'conf','rm'))),
                stringsAsFactors=F)
df$Cluster <- as.character(df$Cluster)
df

Cluster,X24,X23,X22,X21,X20,pval,perc,lam,conf,rm
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


### Table results  

In [9]:
set.seed(seeds[1])
res1 <- foreach(t = 0:length(types), .combine=function(x,y) bind_rows(as.data.frame(x),as.data.frame(y)),.packages=c('dplyr','spatstat','poolr')) %dopar% {
    if (t == 0){
        q <- df
    }
    else{
        pp<-create_pp(test, NULL,sections[[t]], types[t])
        rm<-min(as.numeric(quantile(do.call(c,lapply(pp, nndist)),0.5)),0.06) 
        ps<-get_pp_2d(pp, rm, TRUE)
        ## write into data frame
        q <- calc_pp_2d(types[t],pp,ps,sections[[t]])
        q
    }
}
res1

Cluster,X24,X23,X22,X21,X20,pval,perc,lam,conf,rm
<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0003 CLA-EPd-CTX Car3 Glut_1,NA,NA,NA,0.935,0.010,0.05300,9.25,198,0.78,31
0004 CLA-EPd-CTX Car3 Glut_1,NA,0.055,NA,0.445,NA,0.12000,3.75,103,0.84,41
0007 IT EP-CLA Glut_1,NA,NA,NA,NA,0.030,0.03000,3.77,518,0.90,18
0026 L5/6 IT TPE-ENT Glut_2,NA,0.030,NA,NA,NA,0.03000,0.00,133,0.70,40
0061 L5 IT CTX Glut_3,NA,NA,NA,NA,0.005,0.00500,0.00,271,0.78,28
0079 L4/5 IT CTX Glut_2,0.020,NA,NA,0.005,0.005,0.00006,11.68,1060,0.74,7
0086 L4/5 IT CTX Glut_3,0.090,0.015,NA,NA,NA,0.01000,0.00,112,0.84,44
0092 L4/5 IT CTX Glut_4,0.005,0.050,NA,NA,NA,0.00230,0.00,243,0.71,40
0114 L2/3 IT CTX Glut_3,0.005,NA,NA,NA,NA,0.00500,3.08,643,0.79,19


In [10]:
write.csv(res1, "./temp/table-ex1.csv",row.names = FALSE)

Iterate the process for 5 different seeds.

In [67]:
res1<-read.csv("./temp/table-ex1.csv")
res2<-read.csv("./temp/table-ex2.csv")
res3<-read.csv("./temp/table-ex3.csv")
res4<-read.csv("./temp/table-ex4.csv")
res5<-read.csv("./temp/table-ex5.csv")

res<-Reduce("+", list(res1[,2:6],res2[,2:6],res3[,2:6],res4[,2:6],res5[,2:6])) / 5
res<-data.frame(Cluster=res1$Cluster, res)
res$pval<-pv
res$perc<-res1$perc
res$lam<-res1$lam
res$conf<-res1$conf
res$rm<-res1$rm
res

Cluster,X24,X23,X22,X21,X20,pval,perc,lam,conf,rm
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0003 CLA-EPd-CTX Car3 Glut_1,NA,NA,NA,0.922,0.008,4.4e-02,9.25,198,0.78,31
0004 CLA-EPd-CTX Car3 Glut_1,NA,0.064,NA,0.481,NA,1.4e-01,3.75,103,0.84,41
0007 IT EP-CLA Glut_1,NA,NA,NA,NA,0.027,2.7e-02,3.77,518,0.90,18
0026 L5/6 IT TPE-ENT Glut_2,NA,0.015,NA,NA,NA,1.5e-02,0.00,133,0.70,40
0061 L5 IT CTX Glut_3,NA,NA,NA,NA,0.005,5.0e-03,0.00,271,0.78,28
0079 L4/5 IT CTX Glut_2,0.013,NA,NA,0.005,0.005,4.1e-05,11.68,1060,0.74,7
0086 L4/5 IT CTX Glut_3,0.058,0.014,NA,NA,NA,6.6e-03,0.00,112,0.84,44
0092 L4/5 IT CTX Glut_4,0.005,0.067,NA,NA,NA,3.0e-03,0.00,243,0.71,40
0114 L2/3 IT CTX Glut_3,0.005,NA,NA,NA,NA,5.0e-03,3.08,643,0.79,19


In [68]:
write.csv(res, "table-ex.csv",row.names = FALSE)